In [1]:
import xarray as xr
import os
import tensorflow as tf
from util import *   # ./util.py
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np

print('TensorFlow version:',tf.__version__)

TensorFlow version: 2.10.0


In [3]:
variables = [
        'lat1',
        'lon1',
        'zenith_angle',
        'azimuth_angle',
        'skt',
        'u10n',
        'v10n',
        'u10',
        'v10',
        'sp',
        'msl',
        'fsr',
        'u100',
        'v100',
        'cvl',
        'cvh',
        'tvl',
        'tvh',
        'swvl1',
        'swvl2',
        'swvl3',
        'swvl4',
        'slt',
        'lai_lv',
        'lai_hv',
        'sdfor',
        'stl1',
        'stl2',
        'stl3',
        'stl4',
        'd2m'
    ]

x, dT = load_data(variables)

logging atms files:
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_00.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_03.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_06.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_09.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_12.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_15.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_18.nc
.\data\atms\atms_n20_v2_4_1_collocated_era5_atms_n20_20180614_21.nc
logging model files:
.\data\model\collocated_era5_atms_n20_20180614_00.nc
.\data\model\collocated_era5_atms_n20_20180614_03.nc
.\data\model\collocated_era5_atms_n20_20180614_06.nc
.\data\model\collocated_era5_atms_n20_20180614_09.nc
.\data\model\collocated_era5_atms_n20_20180614_12.nc
.\data\model\collocated_era5_atms_n20_20180614_15.nc
.\data\model\collocated_era5_atms_n20_20180614_18.nc
.\data\model\collocated_era5_atms_n20_201806

In [5]:
x_train, x_test, y_train, y_test = split_normalize_data(x, dT, variable_names=variables, method='lon1')

In [6]:
# find the best variables in a shallow network
best_variables = []

remaining_variables = list(np.arange(x_train.shape[1]))
shallow_losses = {}

while len(best_variables) < x_train.shape[1]:
#while len(best_variables) < 2:    
    
    best_var = -1
    best_loss = 999999999
    
    for var in remaining_variables:
        current_variables = sorted(best_variables + [var])
        print('training with variables', str(current_variables))
        
        x_train_subset = x_train[:,current_variables]
        x_test_subset = x_test[:,current_variables]
        
        model = tf.keras.models.Sequential([
            tf.keras.layers.Dense(x_train_subset.shape[1], activation='relu'),
            tf.keras.layers.Dense(22)
        ])

        model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
            filepath='checkpoints\\shallow_BWAHH_' + str(current_variables) + '.h5',
            save_weights_only=True,
            save_best_only=True,
            verbose=0
        )

        loss_fn = tf.keras.losses.MeanSquaredError()

        model.compile(optimizer='adam',
                      loss=loss_fn)

        hist = model.fit(x_train_subset, y_train, 
                  epochs=50,
                  validation_data=(x_test_subset, y_test),
                  callbacks=[model_checkpoint])
        
        current_loss = min(hist.history['val_loss'])
        losses[str(current_variables)] = current_loss
        
        if current_loss < best_loss:
            best_var = var
            best_loss = current_loss
    
    print('Best variable:', best_var)
    best_variables.append(best_var)
    remaining_variables.remove(best_var)
    

training with variables [0]
Epoch 1/50
1122/1122 [==============================] - 2s 2ms/step - loss: 894.4507 - val_loss: 1002.9872
Epoch 2/50
1122/1122 [==============================] - 2s 2ms/step - loss: 805.5181 - val_loss: 780.0279
Epoch 3/50
1122/1122 [==============================] - 2s 2ms/step - loss: 715.9653 - val_loss: 608.5842
Epoch 4/50
1122/1122 [==============================] - 2s 1ms/step - loss: 658.7767 - val_loss: 507.3776
Epoch 5/50
 349/1122 [========>.....................] - ETA: 1s - loss: 636.5125

KeyboardInterrupt: 